# FastSAM

In [69]:
import numpy as np
model_path = '/home/vishal/Volume_E/Active/Gap_year/research/3d_manipulation/codes/intrinsic_image_decomposition/FastSAM/weights/FastSAM-s.pt'
img_path = '/home/vishal/Volume_E/Active/Gap_year/research/3d_manipulation/codes/intrinsic_image_decomposition/test_images/3.jpeg'
img_path = '/home/vishal/Volume_E/Active/Gap_year/research/3d_manipulation/codes/intrinsic_image_decomposition/test_images/drawer2.jpg'
# bbox_prompt = [[110, 155, 130, 180]]
bbox_prompt = np.round(np.array([
    [176.14-2, 160.6-2, 194.09+2, 179.04+2],
    [60.77-2, 168.81-2, 76.15+2, 186.3+2],
    [74.76-2, 189.06-2, 92.14+2, 210.0+2],
    [171.51-2, 213.61+2, 189.29+2, 236.14-2]
])).tolist()

In [70]:
import sys
import os
import ast
import torch
from PIL import Image

sys.path.append('/home/vishal/Volume_E/Active/Gap_year/research/3d_manipulation/codes/intrinsic_image_decomposition/FastSAM')

from fastsam import FastSAM, FastSAMPrompt 
from utils.tools import convert_box_xywh_to_xyxy

In [60]:
model = FastSAM(model_path)

/home/vishal/Volume_E/Active/Gap_year/research/3d_manipulation/codes/intrinsic_image_decomposition/FastSAM/ultralytics/nn/tasks.py:518: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental fe

In [71]:
input = Image.open(img_path)
input = input.convert("RGB")
import numpy as np
np.array(input).shape

(1080, 810, 3)

In [72]:
everything_results = model(
        input,
        device='cuda',
        retina_masks=True, # For drawing high resolution segmentation masks
        imgsz=256, # Image size (on terminal, it directly worked with 1024. Therefore, try with both 256 and 1024)
        conf=0.4, # Object confidence threshold; default - 0.4
        iou=0.9 # IOU thresholding for filtering annotations    
        )


0: 256x192 18 objects, 56.2ms
Speed: 1.7ms preprocess, 56.2ms inference, 11.5ms postprocess per image at shape (1, 3, 256, 256)


In [73]:
everything_results_1024 = model(
        input,
        device='cuda',
        retina_masks=True, # For drawing high resolution segmentation masks
        imgsz=1024, # Image size (on terminal, it directly worked with 1024. Therefore, try with both 256 and 1024)
        conf=0.4, # Object confidence threshold; default - 0.4
        iou=0.9 # IOU thresholding for filtering annotations    
        )


0: 1024x768 38 objects, 59.1ms
Speed: 4.6ms preprocess, 59.1ms inference, 17.8ms postprocess per image at shape (1, 3, 1024, 1024)


In [74]:
prompt_process = FastSAMPrompt(input, everything_results_1024, 'cudaSeg')
prompt_process.everything_prompt()

tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0., 

In [64]:
prompt_process = FastSAMPrompt(input, everything_results, 'cudaSeg')
ann = prompt_process.box_prompt(bboxes=bbox_prompt)
bboxes = bbox_prompt


In [76]:
prompt_process.plot(
        annotations=ann,
        output_path='/home/vishal/Volume_E/Active/Gap_year/research/3d_manipulation/codes/intrinsic_image_decomposition/results/fastsam/drawer2_1024.jpg',
        bboxes = None,
        points = None,
        point_label = None,
        withContours=True,
        better_quality=True,
    )

In [50]:
prompt_process.plot(
        annotations=ann,
        output_path='/home/vishal/Volume_E/Active/Gap_year/research/3d_manipulation/codes/intrinsic_image_decomposition/results/fastsam/3.jpeg',
        bboxes = bboxes,
        points = None,
        point_label = None,
        withContours=False,
        better_quality=True,
    )

In [56]:
# type(ann)
print(ann.shape) # instance wise mask
net_ann = ann.sum(axis=0)
print(net_ann.shape, net_ann.min(), net_ann.max())

(4, 256, 256)
(256, 256) 0.0 1.0


In [53]:
np.save('/home/vishal/Volume_E/Active/Gap_year/research/3d_manipulation/codes/intrinsic_image_decomposition/results/fastsam/3_seg.npy', ann)

In [57]:
import cv2
cv2.imwrite('/home/vishal/Volume_E/Active/Gap_year/research/3d_manipulation/codes/intrinsic_image_decomposition/results/fastsam/3_seg.png', net_ann*255)

True